In [1]:
from matplotlib import pyplot as plt
import torch
from PIL import Image
from strhub.data.module import SceneTextDataModule
from strhub.models.parseq.system import PARSeq

/scratch/cluster/abhinavc/miniforge3/envs/d2_parseq/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parseq = PARSeq.load_from_checkpoint("models/str_v3_combined/parseq_str.ckpt")
img_transform = SceneTextDataModule.get_transform(parseq.hparams.img_size)

In [12]:
im_path = "label_reading/mixed_12-31.jpg"

img = Image.open(im_path).convert('RGB')
# Preprocess. Model expects a batch of images with shape: (B, C, H, W)
img = img_transform(img).unsqueeze(0)

logits = parseq(img)
# logits.shape  # torch.Size([1, 26, 95]), 94 characters + [EOS] symbol

# # Greedy decoding
pred = logits.softmax(-1)
label, confidence = parseq.tokenizer.decode(pred)
label = label[0]
label

'^'

In [32]:
from glob import glob
import os

from tqdm import tqdm

correct = 0
total = 0
for im_path in glob("label_reading/*.jpg"):
    # if "mixed" not in im_path:
    #     continue
    img = Image.open(im_path).convert('RGB')
    # Preprocess. Model expects a batch of images with shape: (B, C, H, W)
    img = img_transform(img).unsqueeze(0)

    logits = parseq(img)
    # logits.shape  # torch.Size([1, 26, 95]), 94 characters + [EOS] symbol

    # # Greedy decoding
    pred = logits.softmax(-1)
    label, confidence = parseq.tokenizer.decode(pred)
    label = label[0]
    print(os.path.basename(im_path), " ", label)
    # gt = os.path.splitext(os.path.basename(im_path))[0].split("-")[-1]
    # if label.rstrip().lower() == gt.rstrip().lower():
    #     correct += 1
    # else:
    #     print(f"{os.path.basename(im_path)}, gt: {gt.rstrip().lower()}, pred: {label.rstrip().lower()}")
    
    total += 1

# print(f"correct: {correct}")
# print(f"total: {total}")
# print(f"accuracy: {correct / total * 100:.2f}")

mixed_102-0.jpg   32
mixed_102-1.jpg   34
mixed_102-2.jpg   31
mixed_102-3.jpg   42
mixed_102-4.jpg   33
mixed_102-5.jpg   40
mixed_102-6.jpg   41
mixed_102-7.jpg   38
mixed_102-8.jpg   36
mixed_102-9.jpg   37
mixed_102-10.jpg   35
mixed_102-11.jpg   39
mixed_99-0.jpg   15
mixed_99-1.jpg   14
mixed_99-2.jpg   <<|>
mixed_99-3.jpg   12
mixed_99-4.jpg   *L
mixed_99-5.jpg   P
mixed_99-6.jpg   5
mixed_99-7.jpg   >|<
mixed_99-8.jpg   2
mixed_99-9.jpg   3
mixed_99-10.jpg   10
mixed_99-11.jpg   6
mixed_99-12.jpg   11
mixed_99-13.jpg   4
mixed_99-14.jpg   7
mixed_99-15.jpg   9
mixed_99-16.jpg   8
mixed_55-0.jpg   6
mixed_55-1.jpg   *1
mixed_55-2.jpg   P1
mixed_55-3.jpg   5
mixed_55-4.jpg   P2
mixed_55-5.jpg   >|<
mixed_55-6.jpg   2
mixed_55-7.jpg   4
mixed_55-8.jpg   <|>
mixed_55-9.jpg   3
mixed_55-10.jpg   ^
mixed_19-0.jpg   NB
mixed_19-1.jpg   N3
mixed_19-2.jpg   N1
mixed_19-3.jpg   BR
mixed_19-4.jpg   #
mixed_19-5.jpg   N2
mixed_19-6.jpg   ^
mixed_19-7.jpg   >|<
mixed_19-8.jpg   <|>
mixed_12